In [2]:
import os
import tempfile
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageColor
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip, ColorClip, vfx
import ipywidgets as widgets
from IPython.display import display, HTML, Video
import warnings
import matplotlib.pyplot as plt
import traceback
import string
import random

%matplotlib inline
warnings.filterwarnings('ignore')

def get_font_path():
    system_paths = [
        "/System/Library/Fonts/Supplemental/Arial.ttf",
        "/Windows/Fonts/arial.ttf",
        "/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf",
        "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"
    ]
    for path in system_paths:
        if os.path.exists(path):
            return path
    return "Arial"

class JupyterVideoAdGenerator:
    def __init__(self):
        self.video_path = None
        self.temp_video_file = None
        self.video_loaded = False
        self.is_generating = False
        self.font_path = get_font_path()
        self.setup_widgets()
        self.setup_ui()

    def hex_to_rgba(self, hex_color, alpha_float):
        rgb_tuple = ImageColor.getrgb(hex_color)
        return (*rgb_tuple, int(alpha_float * 255))

    def setup_widgets(self):
        self.headline_text = widgets.Text(value='Amazing Product!', description='Headline:', style={'description_width': 'initial'})
        self.description_text = widgets.Text(value='A short statement.', description='Description:', style={'description_width': 'initial'})
        self.callout_text = widgets.Text(value='Shop Now', description='Call-to-Action:', style={'description_width': 'initial'})
        self.show_headline = widgets.Checkbox(value=True, description='Show', indent=False)
        self.show_description = widgets.Checkbox(value=True, description='Show', indent=False)
        self.show_callout = widgets.Checkbox(value=True, description='Show', indent=False)

        bg_styles = ['none', 'card', 'vignette', 'gradient-top', 'gradient-bottom', 'gradient-left', 'gradient-right']
        self.headline_bg_style = widgets.Dropdown(options=bg_styles, value='card', description='H. BG Style:', style={'description_width': 'initial'})
        self.description_bg_style = widgets.Dropdown(options=bg_styles, value='none', description='D. BG Style:', style={'description_width': 'initial'})

        self.headline_font_size = widgets.IntSlider(value=70, min=20, max=150, description='H. Size:')
        self.headline_color = widgets.ColorPicker(value='#FFFFFF', description='H. Text Color:')
        self.headline_outline_color = widgets.ColorPicker(value='#000000', description='H. Outline:')
        self.headline_outline_thickness = widgets.IntSlider(value=1, min=0, max=10, step=1, description='H. O-Thick:')
        self.headline_card_color = widgets.ColorPicker(value='#000000', description='H. Card Color:')
        self.headline_card_opacity = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.05, description='H. Card Opac:')
        self.headline_card_radius = widgets.IntSlider(value=15, min=0, max=50, step=1, description='H. Card Rad:')

        self.description_font_size = widgets.IntSlider(value=40, min=16, max=100, description='D. Size:')
        self.description_color = widgets.ColorPicker(value='#FFFFFF', description='D. Text Color:')
        self.description_outline_color = widgets.ColorPicker(value='#000000', description='D. Outline:')
        self.description_outline_thickness = widgets.IntSlider(value=0, min=0, max=10, step=1, description='D. O-Thick:')
        self.description_card_color = widgets.ColorPicker(value='#000000', description='D. Card Color:')
        self.description_card_opacity = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.05, description='D. Card Opac:')
        self.description_card_radius = widgets.IntSlider(value=15, min=0, max=50, step=1, description='D. Card Rad:')

        self.callout_font_size = widgets.IntSlider(value=35, min=16, max=80, description='CTA Size:')
        self.callout_color = widgets.ColorPicker(value='#FFFFFF', description='CTA Text:')
        self.callout_bg_color = widgets.ColorPicker(value='#FF4444', description='CTA BG:')
        self.callout_button_style = widgets.RadioButtons(options=['filled', 'outline'], value='filled', description='CTA Style:', style={'description_width': 'initial'})
        self.callout_outline_color = widgets.ColorPicker(value='#000000', description='CTA Outline:')
        self.callout_outline_thickness = widgets.IntSlider(value=0, min=0, max=10, step=1, description='CTA O-Thick:')

        self.sequence_type = widgets.RadioButtons(options=['All Together', 'Sequential'], value='Sequential', description='Anim. Seq.:')
        self.text_delay = widgets.FloatSlider(value=0.5, min=0, max=5, step=0.1, description='Initial Delay (s):')
        self.headline_duration = widgets.FloatSlider(value=2.5, min=0.5, max=10, step=0.1, description='Headline Dura:')
        self.description_duration = widgets.FloatSlider(value=3.0, min=0.5, max=10, step=0.1, description='Desc. Dura:')
        self.callout_duration = widgets.FloatSlider(value=4.0, min=0.5, max=10, step=0.1, description='Callout Dura:')
        self.fade_in_duration = widgets.FloatSlider(value=0.3, min=0, max=3, step=0.1, description='FadeIn (s):')
        self.fade_out_duration = widgets.FloatSlider(value=0.3, min=0, max=3, step=0.1, description='FadeOut (s):')

        self.cinematic_bars_style = widgets.Dropdown(options=['None', 'Letterbox 16:9', 'Letterbox 2.35:1'], value='None', description='Cinematic Bars:', style={'description_width': 'initial'})

        self.file_upload = widgets.FileUpload(accept='.mp4,.mov', multiple=False, description='Upload Video')
        self.file_upload.observe(self.on_file_upload, names='value')
        self.manual_video_path = widgets.Text(placeholder='Or enter path', style={'description_width': 'initial'}, layout={'width': '250px'})
        self.manual_video_path.observe(self.on_manual_path_change, names='value')
        self.preview_button = widgets.Button(description='Update Preview', button_style='info', icon='eye')
        self.preview_button.on_click(self.update_preview)
        self.generate_button = widgets.Button(description='Generate Video', button_style='success', icon='cogs')
        self.generate_button.on_click(self.generate_video)
        self.output_filename = widgets.Text(value='styled_ad_video.mp4', description='Output File:', style={'description_width': 'initial'})
        self.progress = widgets.IntProgress(value=0, description='Progress:', bar_style='info')
        self.status_output = widgets.Output()

    def setup_ui(self):
        text_box = widgets.VBox([
            widgets.HBox([self.show_headline, self.headline_text]),
            widgets.HBox([self.show_description, self.description_text]),
            widgets.HBox([self.show_callout, self.callout_text])
        ])
        style_box = widgets.VBox([
            widgets.HTML("<b>Headline Style</b>"),
            widgets.HBox([self.headline_font_size, self.headline_color, self.headline_outline_color, self.headline_outline_thickness]),
            widgets.HBox([self.headline_bg_style, self.headline_card_color, self.headline_card_opacity, self.headline_card_radius]),
            widgets.HTML("<hr style='margin: 5px 0;'><b>Description Style</b>"),
            widgets.HBox([self.description_font_size, self.description_color, self.description_outline_color, self.description_outline_thickness]),
            widgets.HBox([self.description_bg_style, self.description_card_color, self.description_card_opacity, self.description_card_radius]),
            widgets.HTML("<hr style='margin: 5px 0;'><b>Call-to-Action Style</b>"),
            widgets.HBox([self.callout_font_size, self.callout_color, self.callout_outline_color, self.callout_outline_thickness]),
            widgets.HBox([self.callout_bg_color, self.callout_button_style])
        ])
        timing_box = widgets.VBox([
            self.sequence_type, self.text_delay,
            self.headline_duration, self.description_duration, self.callout_duration,
            widgets.HTML("<hr style='margin: 5px 0;'><b>Fade Effects (All Elements)</b>"),
            self.fade_in_duration, self.fade_out_duration
        ])
        video_effects_box = widgets.VBox([self.cinematic_bars_style])
        video_io_box = widgets.VBox([
            widgets.HBox([self.file_upload, self.manual_video_path]),
            self.output_filename,
            widgets.HBox([self.preview_button, self.generate_button]),
            self.progress, self.status_output,
            widgets.HTML("<hr style='margin:10px 0'><b>General Video Effects</b>"),
            video_effects_box
        ])
        self.tabs = widgets.Tab(children=[video_io_box, text_box, style_box, timing_box])
        self.tabs.set_title(0, "📹 Video & Output"); self.tabs.set_title(1, "📝 Text Content")
        self.tabs.set_title(2, "🎨 Styling"); self.tabs.set_title(3, "⏱️ Timing")

    def display_app(self):
        display(widgets.HTML("<h1>✨ Enhanced Video Ad Generator</h1>")); display(self.tabs)

    def on_file_upload(self, change):
        self._cleanup_temp_file()
        with self.status_output:
            self.status_output.clear_output()
        if not change['new']:
            return
        try:
            up_file = change['new'][0]
            name, cont = up_file['name'], up_file['content']
            safe_name = ''.join(c if c in string.ascii_letters + string.digits + '.' else '_' for c in name)
            self.temp_video_file = os.path.join(tempfile.gettempdir(), f"gen_{os.urandom(4).hex()}_{safe_name}")
            with open(self.temp_video_file, 'wb') as f:
                f.write(cont)
            if not os.path.exists(self.temp_video_file):
                raise FileNotFoundError(f"Failed to create temporary file: {self.temp_video_file}")
            self.video_path = self.temp_video_file
            self.video_loaded = True
            with self.status_output:
                print(f"✅ Video '{name}' uploaded to {self.temp_video_file}")
        except Exception as e:
            self.video_loaded = False
            with self.status_output:
                print(f"❌ Upload error: {e}")
                traceback.print_exc()

    def on_manual_path_change(self, change):
        if not self.is_generating:
            self._cleanup_temp_file()
        path = change['new'].strip()
        if path and os.path.exists(path):
            self.video_path = path
            self.video_loaded = True
            with self.status_output:
                print(f"✅ Manual path set: {path}")
        else:
            self.video_loaded = False
            with self.status_output:
                print(f"❌ Invalid path: {path}")

    def _get_font(self, size):
        try:
            return ImageFont.truetype(self.font_path, size)
        except IOError:
            return ImageFont.load_default()

    def _get_text_dimensions(self, draw, text_content, font, outline_thickness=0):
        try:
            bbox = draw.textbbox((0, 0), text_content, font=font, stroke_width=outline_thickness, anchor='lt')
            text_w, text_h = bbox[2] - bbox[0], bbox[3] - bbox[1]
        except TypeError:
            try:
                text_w, text_h = draw.textsize(text_content, font=font, stroke_width=outline_thickness if outline_thickness > 0 else 0)
            except AttributeError:
                text_w, text_h = draw.textsize(text_content, font=font)
        return text_w, text_h

    def _render_text_to_draw_object(self, draw, text_content, position, font, fill_color, outline_color=None, outline_thickness=0):
        text_x, text_y = position
        if outline_thickness > 0 and outline_color and outline_color != (0, 0, 0, 0):
            try:
                draw.text((text_x, text_y), text_content, font=font, fill=fill_color, stroke_width=outline_thickness, stroke_fill=outline_color, anchor='lt')
            except TypeError:
                try:
                    draw.text((text_x, text_y), text_content, font=font, fill=fill_color, stroke_width=outline_thickness, stroke_fill=outline_color)
                except AttributeError:
                    for dx_s in range(-outline_thickness, outline_thickness + 1):
                        for dy_s in range(-outline_thickness, outline_thickness + 1):
                            if dx_s * dx_s + dy_s * dy_s <= outline_thickness * outline_thickness:
                                draw.text((text_x + dx_s, text_y + dy_s), text_content, font=font, fill=outline_color)
                    draw.text((text_x, text_y), text_content, font=font, fill=fill_color)
        else:
            try:
                draw.text((text_x, text_y), text_content, font=font, fill=fill_color, anchor='lt')
            except TypeError:
                draw.text((text_x, text_y), text_content, font=font, fill=fill_color)

    def _create_background_layer(self, bg_style, canvas_width, canvas_height, text_w, text_h, card_color_hex, card_opacity, card_radius):
        bg_img = Image.new('RGBA', (canvas_width, canvas_height), (0, 0, 0, 0))
        bg_draw = ImageDraw.Draw(bg_img)
        text_x, text_y = 0, 0

        if bg_style == 'card':
            pad = 20
            cw, ch = text_w + 2 * pad, text_h + 2 * pad
            cx, cy = (canvas_width - cw) / 2, (canvas_height - ch) / 2
            rgba_fill = self.hex_to_rgba(card_color_hex, card_opacity)
            bg_draw.rounded_rectangle([cx, cy, cx + cw, cy + ch], radius=card_radius, fill=rgba_fill)
            text_x, text_y = cx + pad, cy + pad
        elif 'gradient' in bg_style or 'vignette' in bg_style:
            for i in range(canvas_width):
                for j in range(canvas_height):
                    alpha = 0
                    if 'gradient-bottom' in bg_style:
                        alpha = int(200 * (j / canvas_height))
                    elif 'gradient-top' in bg_style:
                        alpha = int(200 * (1 - j / canvas_height))
                    elif 'gradient-right' in bg_style:
                        alpha = int(200 * (i / canvas_width))
                    elif 'gradient-left' in bg_style:
                        alpha = int(200 * (1 - i / canvas_width))
                    elif 'vignette' in bg_style:
                        dx, dy = (i - canvas_width / 2), (j - canvas_height / 2)
                        dist = np.sqrt(dx * dx + dy * dy)
                        max_dist = np.sqrt((canvas_width / 2) ** 2 + (canvas_height / 2) ** 2)
                        if max_dist > 0:
                            alpha = int(220 * (dist / max_dist) ** 2)
                    if alpha > 0:
                        bg_draw.point((i, j), fill=(0, 0, 0, min(alpha, 255)))
            text_x, text_y = (canvas_width - text_w) / 2, (canvas_height - text_h) / 2
        else:
            text_x, text_y = (canvas_width - text_w) / 2, (canvas_height - text_h) / 2
            return None, text_x, text_y
        return bg_img, text_x, text_y

    def _draw_text_with_style(self, draw, text_content, font, color, bg_style, canvas_width, canvas_height,
                             outline_color=None, outline_thickness=0,
                             card_color_hex='#000000', card_opacity=0.5, card_radius=15):
        text_w, text_h = self._get_text_dimensions(draw, text_content, font, outline_thickness)
        text_x_final, text_y_final = (canvas_width - text_w) / 2, (canvas_height - text_h) / 2
        canvas_img = draw.im
        if bg_style != 'none':
            background_layer_img, text_x_on_bg, text_y_on_bg = self._create_background_layer(
                bg_style, canvas_width, canvas_height, text_w, text_h,
                card_color_hex, card_opacity, card_radius
            )
            if background_layer_img:
                try:
                    canvas_img = Image.new('RGBA', (canvas_width, canvas_height), (0, 0, 0, 0))
                    canvas_img.paste(background_layer_img, (0, 0), background_layer_img)
                    draw = ImageDraw.Draw(canvas_img)
                    text_x_final, text_y_final = text_x_on_bg, text_y_on_bg
                except Exception as e:
                    print(f"Warn: Failed to paste background layer: {e}")
                    text_x_final, text_y_final = text_x_on_bg, text_y_on_bg
        self._render_text_to_draw_object(draw, text_content, (text_x_final, text_y_final), font, color, outline_color, outline_thickness)
        return canvas_img

    def _draw_button(self, draw, text_content, font, text_color, bg_color, button_style_option, canvas_width, canvas_height, outline_color=None, outline_thickness=0):
        if canvas_width <= 0 or canvas_height <= 0:
            print(f"Warning: Invalid button canvas dimensions: {canvas_width}x{canvas_height}")
            return
        rect_bbox = [0, 0, canvas_width, canvas_height]
        if button_style_option == 'filled':
            draw.rounded_rectangle(rect_bbox, radius=15, fill=bg_color)
        else:
            draw.rounded_rectangle(rect_bbox, radius=15, outline=bg_color, width=4)
        text_w, text_h = self._get_text_dimensions(draw, text_content, font, outline_thickness)
        if text_w <= 0 or text_h <= 0:
            print(f"Warning: Invalid text dimensions for button: {text_w}x{text_h}")
            return
        text_pos = ((canvas_width - text_w) / 2, (canvas_height - text_h) / 2)
        self._render_text_to_draw_object(draw, text_content, text_pos, font, text_color, outline_color, outline_thickness)

    def _create_styled_clip(self, video_size, is_button, **kwargs):
        w_vid, h_vid = video_size
        font = self._get_font(kwargs['font_size'])
        txt = kwargs['text']
        img_canvas = None
        canvas_dims = None
        fade_in = kwargs.get('fade_in_duration', 0)
        fade_out = kwargs.get('fade_out_duration', 0)
        measure_draw = ImageDraw.Draw(Image.new('RGBA', (1, 1)))
    
        print(f"Creating clip for {'button' if is_button else 'text'}: '{txt}'")
        if not txt or len(txt.strip()) == 0:
            print(f"Error: Empty text provided for {'button' if is_button else 'text'} clip")
            return None
    
        if is_button:
            actual_text_w, actual_text_h = self._get_text_dimensions(measure_draw, txt, font, kwargs.get('outline_thickness', 0))
            print(f"Button text dimensions: {actual_text_w}x{actual_text_h}")
            if actual_text_w <= 0 or actual_text_h <= 0:
                print(f"Error: Invalid text dimensions for button: {actual_text_w}x{actual_text_h}")
                return None
            px, py = 30, 15
            canvas_dims = (int(actual_text_w + 2 * px), int(actual_text_h + 2 * py))
            if canvas_dims[0] <= 0 or canvas_dims[1] <= 0:
                print(f"Error: Invalid canvas dimensions for button: {canvas_dims}")
                return None
            img_canvas = Image.new('RGBA', canvas_dims, (0, 0, 0, 0))
            draw_obj = ImageDraw.Draw(img_canvas)
            self._draw_button(draw_obj, txt, font, kwargs['color'], kwargs['bg_color'], kwargs['style'], canvas_dims[0], canvas_dims[1], kwargs.get('outline_color'), kwargs.get('outline_thickness', 0))
            position = ('center', h_vid - canvas_dims[1] - 50)  # CTA 50px from bottom
        else:
            canvas_dims = (w_vid, h_vid)
            if canvas_dims[0] <= 0 or canvas_dims[1] <= 0:
                print(f"Error: Invalid canvas dimensions for text: {canvas_dims}")
                return None
            img_canvas = Image.new('RGBA', canvas_dims, (0, 0, 0, 0))
            draw_obj = ImageDraw.Draw(img_canvas)
            img_canvas = self._draw_text_with_style(draw_obj, txt, font, kwargs['color'], kwargs['bg_style'], canvas_dims[0], canvas_dims[1], kwargs.get('outline_color'), kwargs.get('outline_thickness', 0), kwargs.get('card_color_hex'), kwargs.get('card_opacity'), kwargs.get('card_radius'))
            # Stack headline and description vertically
            if kwargs['text'] == self.headline_text.value:
                position = ('center', 50)  # Headline 50px from top
            else:
                position = ('center', 150)  # Description 150px from top
    
        if img_canvas:
            img_array = np.array(img_canvas, dtype=np.uint8)
            print(f"Image array shape for {'button' if is_button else 'text'} '{txt}': {img_array.shape}, position: {position}, start: {kwargs['start']}, duration: {kwargs['duration']}")
            if len(img_array.shape) != 3 or img_array.shape[2] != 4:
                print(f"Error: Invalid image array shape: {img_array.shape}, expected (height, width, 4)")
                return None
            clip = ImageClip(img_array, ismask=False).set_start(kwargs['start']).set_duration(kwargs['duration']).set_position(position)
            if fade_in > 0:
                clip = clip.fadein(fade_in)
            if fade_out > 0:
                clip = clip.fadeout(fade_out)
            return clip
        print(f"Error: Failed to create image canvas for {'button' if is_button else 'text'} '{txt}'")
        return None

    def update_preview(self, button=None):
        if not self.video_loaded:
            with self.status_output:
                self.status_output.clear_output()
                print("❌ Upload video.")
            return
        with self.status_output:
            self.status_output.clear_output()
            print("🎬 Previewing...")
        try:
            with VideoFileClip(self.video_path) as vfc:
                frame_arr = vfc.get_frame(1)
            pil_frame = Image.fromarray(frame_arr)
            preview_clips_params = []
            fade_args = {"fade_in_duration": self.fade_in_duration.value, "fade_out_duration": self.fade_out_duration.value}
            if self.show_headline.value:
                preview_clips_params.append({"start": 0, "duration": 1, **self._get_headline_params(), **fade_args})
            if self.show_description.value:
                preview_clips_params.append({"start": 0, "duration": 1, **self._get_description_params(), **fade_args})
            if self.show_callout.value:
                preview_clips_params.append({"start": 0, "duration": 1, **self._get_callout_params(), **fade_args})

            for p_args in preview_clips_params:
                clip_obj = self._create_styled_clip(pil_frame.size, **p_args)
                if clip_obj:
                    frame_data = clip_obj.get_frame(0)
                    img_obj = Image.fromarray(frame_data)
                    pil_frame.paste(img_obj, (0, 0), img_obj.convert('RGBA'))
            plt.figure(figsize=(12, 7))
            plt.imshow(pil_frame)
            plt.axis('off')
            plt.show()
            with self.status_output:
                print("✅ Preview updated.")
        except Exception as e:
            with self.status_output:
                print(f"❌ Preview error: {e}")
                traceback.print_exc()

    def generate_video(self, button=None):
        if not self.video_loaded:
            with self.status_output:
                self.status_output.clear_output()
                print("❌ Upload video.")
            return
        if not os.path.exists(self.video_path):
            with self.status_output:
                self.status_output.clear_output()
                print(f"❌ Video file not found: {self.video_path}")
            self.video_loaded = False
            return
        with self.status_output:
            self.status_output.clear_output()
            print("🎬 Generating video...")
            self.progress.value = 10
        try:
            self.is_generating = True
            with VideoFileClip(self.video_path) as main_vid_clip:
                print(f"Input video duration: {main_vid_clip.duration}, resolution: {main_vid_clip.size}")
                self.progress.value = 20
                text_clips_to_composite = []
                time_curr = self.text_delay.value
                elems_to_add = []
                fade_params = {"fade_in_duration": self.fade_in_duration.value, "fade_out_duration": self.fade_out_duration.value}
                
                # Print widget settings for debugging
                print(f"Widget settings: show_headline={self.show_headline.value}, show_description={self.show_description.value}, show_callout={self.show_callout.value}")
                print(f"Text values: headline='{self.headline_text.value}', description='{self.description_text.value}', callout='{self.callout_text.value}'")
                print(f"Timing: sequence_type={self.sequence_type.value}, text_delay={self.text_delay.value}, headline_duration={self.headline_duration.value}, description_duration={self.description_duration.value}, callout_duration={self.callout_duration.value}")
    
                if self.show_headline.value and self.headline_text.value.strip():
                    print(f"Adding headline clip: '{self.headline_text.value}'")
                    elems_to_add.append({"duration": self.headline_duration.value, **self._get_headline_params(), **fade_params})
                else:
                    print("Skipping headline clip: not enabled or empty text")
                if self.show_description.value and self.description_text.value.strip():
                    print(f"Adding description clip: '{self.description_text.value}'")
                    elems_to_add.append({"duration": self.description_duration.value, **self._get_description_params(), **fade_params})
                else:
                    print("Skipping description clip: not enabled or empty text")
                if self.show_callout.value and self.callout_text.value.strip():
                    print(f"Adding callout clip: '{self.callout_text.value}'")
                    elems_to_add.append({"duration": self.callout_duration.value, **self._get_callout_params(), **fade_params})
                else:
                    print("Skipping callout clip: not enabled or empty text")
    
                self.progress.value = 40
                for el_params in elems_to_add:
                    s_time = self.text_delay.value if self.sequence_type.value == 'All Together' else time_curr
                    if s_time >= main_vid_clip.duration:
                        print(f"Error: Start time {s_time} exceeds video duration {main_vid_clip.duration} for '{el_params['text']}'")
                        continue
                    el_dura = min(el_params["duration"], main_vid_clip.duration - s_time)
                    if el_dura <= 0:
                        print(f"Error: Invalid duration {el_dura} for '{el_params['text']}'")
                        continue
                    el_params.update({"start": s_time, "duration": el_dura})
                    styled_text_clip = self._create_styled_clip(main_vid_clip.size, **el_params)
                    if styled_text_clip:
                        text_clips_to_composite.append(styled_text_clip)
                        print(f"Successfully added clip for '{el_params['text']}' at start={s_time}, duration={el_dura}, position={el_params['position']}")
                    else:
                        print(f"Error: Failed to create clip for '{el_params['text']}'")
                    if self.sequence_type.value == 'Sequential':
                        time_curr += el_dura
    
                if not text_clips_to_composite:
                    print("Warning: No text clips were created for compositing")
                final_composite_elements = [main_vid_clip] + text_clips_to_composite
                print(f"Compositing {len(final_composite_elements)} clips (1 video + {len(text_clips_to_composite)} text clips)")
                processed_video_with_text = CompositeVideoClip(final_composite_elements, size=main_vid_clip.size)
    
                bar_style = self.cinematic_bars_style.value
                if bar_style != 'None':
                    w, h = processed_video_with_text.size
                    target_aspect = 0
                    if bar_style == 'Letterbox 16:9':
                        target_aspect = 16/9
                    elif bar_style == 'Letterbox 2.35:1':
                        target_aspect = 2.35/1
    
                    if target_aspect > 0:
                        current_aspect = w/h
                        if abs(current_aspect - target_aspect) > 0.01:
                            content_w, content_h = w, h
                            if current_aspect > target_aspect:
                                content_w = int(h * target_aspect)
                            else:
                                content_h = int(w / target_aspect)
                            resized_content = processed_video_with_text.fx(vfx.resize, width=content_w, height=content_h).set_position(('center', 'center'))
                            black_bg = ColorClip(size=(w, h), color=(0, 0, 0), duration=processed_video_with_text.duration)
                            processed_video_with_text = CompositeVideoClip([black_bg, resized_content], size=(w, h))
    
                self.progress.value = 80
                with self.status_output:
                    print("🎞️ Writing file...")
                processed_video_with_text.write_videofile(self.output_filename.value, codec='libx264', audio_codec='aac', verbose=False, logger=None, ffmpeg_params=['-preset', 'fast'])
            self.progress.value = 100
            with self.status_output:
                print(f"✅ Saved: {self.output_filename.value}")
            display(Video(self.output_filename.value, width=640, embed=True))
        except Exception as e:
            with self.status_output:
                print(f"❌ Error: {e}")
                traceback.print_exc()
            self.progress.value = 0
        finally:
            self.is_generating = False
            self._cleanup_temp_file()

    def _get_headline_params(self):
        return {
            "font_size": self.headline_font_size.value,
            "text": self.headline_text.value,
            "color": self.hex_to_rgba(self.headline_color.value, 1.0),
            "bg_style": self.headline_bg_style.value,
            "position": 'center',
            "is_button": False,
            "outline_color": self.hex_to_rgba(self.headline_outline_color.value, 1.0),
            "outline_thickness": self.headline_outline_thickness.value,
            "card_color_hex": self.headline_card_color.value,
            "card_opacity": self.headline_card_opacity.value,
            "card_radius": self.headline_card_radius.value
        }

    def _get_description_params(self):
        return {
            "font_size": self.description_font_size.value,
            "text": self.description_text.value,
            "color": self.hex_to_rgba(self.description_color.value, 1.0),
            "bg_style": self.description_bg_style.value,
            "position": 'center',
            "is_button": False,
            "outline_color": self.hex_to_rgba(self.description_outline_color.value, 1.0),
            "outline_thickness": self.description_outline_thickness.value,
            "card_color_hex": self.description_card_color.value,
            "card_opacity": self.description_card_opacity.value,
            "card_radius": self.description_card_radius.value
        }

    def _get_callout_params(self):
        return {
            "font_size": self.callout_font_size.value,
            "text": self.callout_text.value,
            "color": self.hex_to_rgba(self.callout_color.value, 1.0),
            "bg_color": self.hex_to_rgba(self.callout_bg_color.value, 1.0),
            "style": self.callout_button_style.value,
            "position": 'bottom_center',
            "is_button": True,
            "outline_color": self.hex_to_rgba(self.callout_outline_color.value, 1.0),
            "outline_thickness": self.callout_outline_thickness.value
        }

    def _cleanup_temp_file(self):
        if self.temp_video_file and os.path.exists(self.temp_video_file) and not self.is_generating:
            try:
                os.remove(self.temp_video_file)
                with self.status_output:
                    print(f"🗑️ Cleaned up temporary file: {self.temp_video_file}")
                self.temp_video_file = None
            except OSError as e:
                with self.status_output:
                    print(f"⚠️ Could not clean up temporary file: {e}")

generator = JupyterVideoAdGenerator()
generator.display_app()

HTML(value='<h1>✨ Enhanced Video Ad Generator</h1>')